In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.sparse import csr_matrix

from gtda.graphs import *
from gtda.homology import *
from gtda.diagrams import *

from igraph import Graph, plot

from IPython.display import SVG, display

In [ ]:
def make_directed_circle_adjacency(n_vertices):
    weights = np.ones(n_vertices)
    rows = np.arange(n_vertices)
    columns = np.arange(1, n_vertices + 1) % n_vertices
    return csr_matrix((weights, (rows, columns)))

n_vertices = 4
directed_circle = make_directed_circle_adjacency(n_vertices)

# Attempt with `FlagserPersistence`

In [ ]:
row, col = directed_circle.nonzero()
graph = Graph(n=n_vertices, edges=list(zip(row, col)), directed=True)
# plot(graph, layout='kk')
fname = "graph.svg"
graph.write_svg(fname, layout='kk')
display(SVG(filename=fname))

In [ ]:
directed_circle_gd = GraphGeodesicDistance(directed=True).fit_transform([directed_circle])
undirected_circle_gd = GraphGeodesicDistance(directed=False).fit_transform([directed_circle])
# graph_gd = Graph.Adjacency(directed_circle_gd[0].tolist()))
# plot(graph_gd, layout="kk")

In [ ]:
FP = FlagserPersistence(directed=True)
FP.fit_transform_plot(directed_circle_gd)

In [ ]:
FP = FlagserPersistence(directed=False)  # Alternative FP = VietorisRipsPersistence(metric="precomputed")
FP.fit_transform_plot(undirected_circle_gd)

In [ ]:
rows_flipped = np.concatenate([row[::2], col[1::2]])
columns_flipped = np.concatenate([col[::2], row[1::2]])
weights = np.ones(n_vertices)
directed_circle_flipped = csr_matrix((weights, (rows_flipped, columns_flipped)), shape=(n_vertices, n_vertices))

graph = Graph(n=n_vertices, edges=list(zip(rows_flipped, columns_flipped)), directed=True)
# plot(graph, layout="kk")
fname = "graph.svg"
graph.write_svg(fname, layout='kk')
display(SVG(filename=fname))

In [ ]:
directed_circle_flipped_gd = GraphGeodesicDistance(directed=True).fit_transform([directed_circle_flipped])
FP = FlagserPersistence(directed=True)
FP.fit_transform_plot(directed_circle_flipped_gd)

# Figure-eight experiments

In [ ]:
N = 10
M = 20

fig_8 = np.full((N + M - 1, N + M - 1), fill_value=np.inf)
fig_8[range(N - 1), range(1, N)] = 1
fig_8[N - 1, 0] = 1
fig_8[range(N - 1, N + M - 2), range(N, N + M - 1)] = 1
fig_8[N + M - 2, N - 1] = 1
np.fill_diagonal(fig_8, 0)

graph = Graph.Adjacency(fig_8.tolist())
layout = graph.layout("kk")
plot(graph, layout=layout)

In [ ]:
fig_8_gd = GraphGeodesicDistance(directed=True).fit_transform([fig_8])
FP = FlagserPersistence()
FP.fit_transform_plot(fig_8_gd);

## Undirected case

In [ ]:
N = 10
M = 20

fig_8 = np.full((N + M - 1, N + M - 1), fill_value=0)
fig_8[range(N - 1), range(1, N)] = 1
fig_8[N - 1, 0] = 1
fig_8[range(N - 1, N + M - 2), range(N, N + M - 1)] = 1
fig_8[N + M - 2, N - 1] = 1

fig_8 = fig_8 + fig_8.T
fig_8 = np.where(fig_8, fig_8 != 0, np.inf)

np.fill_diagonal(fig_8, 0)

graph = Graph.Adjacency(fig_8.tolist())
layout = graph.layout("kk")
plot(graph, layout=layout)

In [ ]:
fig_8_gd = GraphGeodesicDistance(directed=False).fit_transform([fig_8])
VR = VietorisRipsPersistence(metric='precomputed')
VR.fit_transform_plot(fig_8_gd);